In [ ]:
import cv2
import imageio
import numpy as np
import plotly.express as px
from skimage.morphology import skeletonize

def load_image(idx=0):
    vid = imageio.get_reader("Ludosport_birthday_sparring.mp4",  'ffmpeg')
    vid_iter = vid.iter_data()
    for _ in range(idx+1):
        image = next(vid_iter)
    return image

test_frames = (51, 56, 78, 104, 174, 454)
img = load_image(test_frames[-1])

px.imshow(img)

In [ ]:
# reload image to ensure reset when running cell
idx = test_frames[-1]
img = load_image(idx)

# convert to HSV for simplicity
blur = cv2.GaussianBlur(img, (3, 3), 0)
kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
blur = cv2.filter2D(blur, ddepth=-1, kernel=kernel)
m1 = blur[:, :, 1] > 200
m2 = blur[:, :, 2] > 200
m3 = blur[:, :, 0] > 240
mask = (m1 + m2 + m3).astype(np.uint8)

rho = 1                         # distance resolution in pixels of the Hough grid
theta = np.pi / 180             # angular resolution in radians of the Hough grid
threshold = 40                  # minimum number of votes (intersections in Hough grid cell)
min_line_length = 25            # minimum number of pixels making up a line
max_line_gap = 10               # maximum gap in pixels between connectable line segments

# Run Hough on edge detected image
# Output "lines" is an array containing endpoints of detected line segments
lines = cv2.HoughLinesP(mask, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)

gray = np.zeros(img.shape[:2], np.uint8)
if isinstance(lines, np.ndarray):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(gray, (x1, y1), (x2, y2), 255, 2)
            if 100 > length > 40:
                x_diff = x1 - x2
                y_diff = y1 - y2
                centroid = (int((x1 + x2) / 2), int((y1 + y2) / 2))
                cv2.drawMarker(img, centroid, (0, 255, 0),
                               markerType=cv2.MARKER_CROSS, thickness=2)

# contours, _ = cv2.findContours(gray, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# for i, contour in enumerate(contours):
#     if 800 > cv2.contourArea(contour) > 60:
#         cv2.drawContours(img, contours, i, 255, -1)
        
px.imshow(img)

## Reconstruct image from SVD
* Simple Hough line tracking can't identify blurred lines
* Sharpening kernels have failed to 

In [ ]:
def reconstruct(img, sv_num=200, start_idx=0, low_thresh=None, up_thresh=None):
    img__ = img.copy()
    img_ = cv2.cvtColor(img__, cv2.COLOR_BGR2GRAY)
    scaled_img = (img_ - img_.mean()) / img_.std()
    U, s, V = np.linalg.svd(scaled_img)
    reconstruction = np.matrix(U[:, start_idx:sv_num]) * np.diag(s[start_idx:sv_num]) * np.matrix(V[start_idx:sv_num, :])
    if low_thresh != None:
        mask = np.logical_and(up_thresh > reconstruction, reconstruction > low_thresh)
        mask = mask.astype(np.uint8)
        cnts, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        for i,c in enumerate(cnts):
            area = cv2.contourArea(c)
            perim = cv2.arcLength(c, True)
            if 1200 > area > 600:# and 300 > perim > 200:
                cv2.drawContours(img__, cnts, i, (255, 0, 0), -1)
        fig = px.imshow(img__)
    else:
        fig = px.imshow(reconstruction)
    fig.show()
    
index = test_frames[-1]
img = load_image(index)
reconstruct(img, 50, 5, 0.8, 1.5)

In [ ]:
px.imshow(img)